In [1]:
# Naive Bayes
import pandas as pd
import numpy as np
import math 
import random
import csv

In [2]:
heart = pd.read_csv("D:\Fall 2020 courses\CSC5825 machine learning\homework\HW2\heart.csv")
heart.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
# Normalizing data using function F(X) = (X 􀀀 mean)=std
heart[['age','trestbps','chol','thalach','oldpeak']] = heart[['age','trestbps','chol','thalach','oldpeak']].apply(lambda data: abs(data - np.mean(data)) / np.std(data), axis = 0)

In [4]:
heart.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.952197,1,3,0.763956,0.256334,1,0,0.015443,0,1.087338,0,0,1,1
1,1.915313,1,2,0.092738,0.072199,0,1,1.633471,0,2.122573,0,0,2,1
2,1.474158,0,1,0.092738,0.816773,0,0,0.977514,0,0.310912,2,0,2,1
3,0.180175,1,1,0.663867,0.198357,0,1,1.239897,0,0.206705,2,0,2,1
4,0.290464,0,0,0.663867,2.082050,0,1,0.583939,1,0.379244,2,0,2,1


In [5]:
heart.shape

(303, 14)

In [6]:
print(list(heart))

['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']


In [7]:
# Separate by class
# creating a dictionary of classes with mean and std of each columns.
def separateUnderclass(data):
    sepclass = {}
    #calcualting mean and std for each attributes excluding target 
    sepclass['1'] = data.loc[data['target'] == 1, data.columns != 'target']
    sepclass['0'] = data.loc[data['target'] == 0, data.columns != 'target']
    return sepclass

In [8]:
#summarizing under each class
def summarizeUnderclass(data):
    info = {}
    sepclass = separateUnderclass(data)
    for classValue, instances in sepclass.items():
        info[classValue] = list(zip(instances.mean(axis =0), instances.std(axis = 0)))
    return info

In [9]:
# calculating gaussian probability density function
def calculateGp(x, mean, stddev):
    expo = math.exp(-(math.pow(x - mean,2) / (2 * math.pow(stddev,2))))
    return(1 / (math.sqrt(2*math.pi) * stddev)) * expo
#     return abs(x - mean) / stddev
#     if stddev != 0:
#         return abs(x - mean) / stddev
#     return mean

In [10]:
# calulating class probabilities
def calculateCp(info,test):
    probabilities = {}
    for classValue, classSummaries in info.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean,stddev = classSummaries[i]
            x = test[i]
            probabilities[classValue] = probabilities[classValue] * calculateGp(x, mean, stddev)
    return probabilities

In [11]:
# making prediction
def predict_probablity(info,test):
    probabilities = calculateCp(info,test)
    # return the class with max probability
    return max(probabilities, key = probabilities.get)

In [12]:
#return predictions 
def getPredictions(summaries,test):
    predictions = []
    for i in test.values.tolist():
        result = predict_probablity(summaries,i)
        predictions.append(int(result))
    return predictions

In [13]:
# Accuracy score
def accuracy_rate(test, predictions):
    correct = 0
    test = test.values.tolist()
    for i in range(len(test)):
        if test[i][-1] == float(predictions[i]):
            correct += 1
    return (correct / float(len(test))) * 100.0

In [14]:
train, test = np.split(heart.sample(frac=1), [int(.8*len(heart))])
print('heart_dataset = {0} , train = {1}, test = {2}'.format(len(heart), len(train), len(test)))
summaries = summarizeUnderclass(train)
predictions = getPredictions(summaries,test)
print("predictions = {0} ".format(predictions))
accuracy = accuracy_rate(test, predictions)
print('Accuracy = {0}%'.format(accuracy))

heart_dataset = 303 , train = 242, test = 61
predictions = [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0] 
Accuracy = 86.88524590163934%


In [15]:
# CONFUSION METRIx
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#target = test['target'].to_list()
confusion_matrix(predictions, test['target'])

array([[17,  2],
       [ 6, 36]], dtype=int64)

In [16]:
accuracy = accuracy_score(predictions, test['target'])
print('Accuracy = {0}%'.format(accuracy))

Accuracy = 0.8688524590163934%


In [17]:
precision = precision_score(predictions, test['target'])
print('Precision = {0}%'.format(precision))

Precision = 0.9473684210526315%


In [18]:
recall = recall_score(predictions, test['target'])
print('recall = {0}%'.format(recall))

recall = 0.8571428571428571%


In [19]:
f1 = f1_score(predictions, test['target'])
print('f1 = {0}%'.format(f1))

f1 = 0.9%
